In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np
import random
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate(SparkConf())
spark = SparkSession.builder.getOrCreate()

In [2]:
rdd = sc.parallelize(range(100))

In [3]:
# helper function
def flatten(x,temp = []):
    '''
    flatten tuple 
    '''
    if type(x) == int or type(x) == float:
        return x
    for val in x:
        value = flatten(val,temp)
        if type(value)== int or type(value) == float:
            temp.append(value)
    return temp

In [18]:
# average
avg = rdd.sum() / rdd.count()
print('Averge :' ,avg)

# median
temp = rdd.sortBy(lambda x :x , ascending = False).zipWithIndex().map(lambda x: x[::-1] )       
n = temp.count()
if n% 2 ==1:
    median = temp.lookup((n-1)/2)[0]
if n% 2 ==0:
    median = (temp.lookup(n/2)[0] + temp.lookup(n/2 -1)[0])/2
print('Median :',median)

# Standard Deviation
N = rdd.count()
std = np.sqrt( 1/N *  rdd.map(lambda x : (x-avg)**2).sum()  )
print('Standard Deviation :',std) # we can be using median+standard deviation to calculate the threshold

# Skewness :
# Negative values for the skewness indicate data that are skewed left and positive values for the skewness
# indicate data that are skewed right. By skewed left, we mean that the left tail is long relative to the
# right tail. Similarly, skewed right means that the right tail is long relative to the left tail.
skew = 1/N * rdd.map(lambda x : ((x - avg)**3)/std**3  ).sum()
print('Skewness :', skew)

# Kurtosis : 
# Reports on the shape of the data when ploted using a histogram, and the outlier content within the data
# in other words, the length of the tails of the histogram.
# the higher the Kurtosis the more outliers are present and the longer the tails of the distribution in the histogram are
kurtosis = 1/N * rdd.map(lambda x : (x-avg)**4/std**4 ).sum()
print('Kurtosis :', kurtosis)

# Correlation
rdd2 = rdd.map(lambda x : x**2 + 30) # creating new values
avg2 = rdd2.sum() / N
std2 = np.sqrt( 1/N *  rdd2.map(lambda x : (x-avg2)**2).sum()  )
covariance = 1/N * rdd.zip(rdd2).map(lambda x : (x[0]-avg)* (x[1]-avg2) ).sum()
correlation = covariance / (std*std2 )
print('Correlation :', correlation)

# Correlation Matrix
# creating rdds 
new_ = rdd
for i in range(4):
    rnd = random.randint(0,100)
    globals()['rdd'+str(i)] = rdd.map(lambda x : x**i +rnd )
    temp = globals()['rdd'+str(i)]
    new_ = new_.zip(temp)
    #print('rdd'+str(i),True)
from pyspark.mllib.stat import Statistics
data = new_.map(lambda x : flatten(x,temp = []) )
print('Correlation Matrix :\n',Statistics.corr(data))

Averge : 49.5
Median : 49.5
Standard Deviation : 28.86607004772212
Skewness : 1.4210854715202004e-16
Kurtosis : 1.7997599759975997
Correlation : 0.9676443927131408
Correlation Matrix :
 [[1.                nan 1.         0.96764439 0.91551566]
 [       nan 1.                nan        nan        nan]
 [1.                nan 1.         0.96764439 0.91551566]
 [0.96764439        nan 0.96764439 1.         0.98595001]
 [0.91551566        nan 0.91551566 0.98595001 1.        ]]


In [23]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
N = rdd.count()
avg = rdd.sum() / N
std = np.sqrt( 1/N *  rdd.map(lambda x : (x-avg)**2).sum()  )
rdd2 = sc.parallelize([7,6,5,4,5,6,7,8,9,10])
avg2 = rdd2.sum() / N
std2 = np.sqrt( 1/N *  rdd2.map(lambda x : (x-avg2)**2).sum()  )
1/N * rdd.zip(rdd2).map(lambda x : (x[0]-avg)* (x[1]-avg2) ).sum() 

3.6500000000000004